In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

Як було визначено під час розробки моделі, через велику кількість нулів у цільовій змінній ми відразу , головну метрику ми обирали поміж RMSE та MAE. Основною їх різницею є те, що RMSE штрафує за більші помилки сильніше, а MAE легше інтерпретується. Було вирішено, що особливість RMSE може бути корисною в контексті задачі. 

In [3]:
categories = ['country_code', 'platform', 'media_source']

non_decreasing_cum_cols_start_with = [
    #day
    'total_sessions_day', 'chapters_finished_day', 'chapters_opened_day',
    'chapters_closed_day', 'diamonds_received_day', 'diamonds_spent_day',
    'tickets_spent_day',
    #session
    'chapters_finished_session', 'chapters_opened_session',
    'chapters_closed_session', 'diamonds_spent_session', 'tickets_spent_session',
    #incomes
    'app_sub_ltv_day', 'app_iap_ltv_day', 'ad_ltv_day'
]

def divide_df(df_t):
    target_cols = [col for col in df_t.columns if col.startswith('target')]
    targets_df = df_t[target_cols]
    df_t.drop(columns=target_cols, inplace=True)
    return targets_df

def generate_differences(df_t):
    for el in non_decreasing_cum_cols_start_with:
        arr = [col[-1] for col in df_t.columns if col.startswith(el)]

        for i in range(len(arr)):
            for j in range(i + 1, len(arr)):
                df_t[el + arr[j] + '-' + el + arr[i]] = df_t[el + arr[j]] - df_t[el + arr[i]]

    return df_t

In [4]:
DATA_PATH = 'tt.zip' ##enter_path_for_data_csv
MODEL_PATH = 'best_model'
data = pd.read_csv(DATA_PATH)
data = data.dropna()
data.drop(columns=['install_date','country_code'], inplace=True)
data_targets = divide_df(data)
data = generate_differences(data)
data= pd.get_dummies(data)

In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(task_type='GPU')
model.load_model(MODEL_PATH)

In [ ]:
preds = model.predict(data)

print(f'Train RMSE: {np.sqrt(mean_squared_error(data, preds))} '
      f'MAE: {mean_absolute_error(data, preds)}')